In [1]:
import time, os, socket, threading
from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import \
ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
import cv2

In [3]:
# chorme_options = webdriver.ChromeOptions()
# chorme_options.add_argument('--headless')
# chorme_options.add_argument('--no-sandbox')
# chorme_options.add_argument('--disable-dev-shm-usage')
# webbrowser를 띄우지 않을때
#search = "coffee" # 검색 키워드
chromedriver_path = '../chromedriver/chromedriver' # 드라이버 파일 위치

# Google Image
## def Func

In [1]:
def create_folder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
def scroll_down() :
    scroll_count = 0
    print("[scroll_down() : 스크롤 다운 시작]")
    
    last_height = driver.execute_script('return document.body.scrollHeight')
    after_click = False
    after_click_exception_span = False
    while True :
        print(f"[스크롤 다운: {scroll_count}]")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scroll_count += 1
        time.sleep(1)
        
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if last_height == new_height :
            try : 
                if driver.find_element('xpath', '//*[@id="islmp"]/div/div/div/div[2]/span') != 0 :
                    more_span = driver.find_element('xpath', '//*[@id="islmp"]/div/div/div/div[2]/span')
                    more_span.click()
                    print('고약한 구글')
            except NoSuchElementException as e :
                print(e)
                
            if after_click is True :
                break
            else :
                try :
                    more_button = driver.find_element('xpath', '//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input')
                    if more_button.is_displayed():
                        more_button.send_keys(Keys.ENTER)
                        after_click = True
                    
                except NoSuchElementException as e:
                    # scroll button이 없을때
                    print(e)
                    break
        last_height = new_height
        
def click_and_save(dir_name, index, img, img_list_length):
    global scraped_count
    
    try:
        img.click()
        driver.implicitly_wait(5) # 5초 기다림
        xpath = '//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[3]/div/a/img'
        src = driver.find_element('xpath', xpath).get_attribute('src')
#         src = driver.find_element(By.CSS_SELECTOR, '.n3VNCb.KAlRDb').get_attribute('src')
        if src.split('.')[-1] == 'png':
            urlretrieve(src, dir_name+'/'+str(scraped_count+1)+".png")
            print(f"{index+1}/{img_list_length} PNG 이미지 저장")
        else :
            urlretrieve(src, dir_name+'/'+str(scraped_count+1)+".jpg")
            print(f"{index+1}/{img_list_length} JPG 이미지 저장")

        scraped_count += 1
    except HTTPError as e :
        print(e)
    except Exception as e :
        print(e)
        return
    
def scraping(dir_name, query):
    global scraped_count

    url = f"https://www.google.com/search?q={query}&tbm=isch&ved=2ahUKEwi037baoc76AhVEyIsBHQnWDywQ2-cCegQIABAA&oq=f&gs_lcp=CgNpbWcQARgAMgQIIxAnMgQIIxAnMggIABCABBCxAzIFCAAQgAQyCAgAEIAEELEDMgUIABCABDIICAAQgAQQsQMyBQgAEIAEMgUIABCABDIFCAAQgARQxwJYxwJggxJoAHAAeACAAYYBiAHhAZIBAzEuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=4C5AY7TvMMSQr7wPiay_4AI&bih=969&biw=1920&rlz=1C5CHFA_enKR1026KR1026#imgrc=X6d1rGBwHMhXsM"
    driver.get(url)
    driver.maximize_window()
    

    scroll_down()
    print('[스크롤 완료]')
    driver.execute_script("window.scrollTo(0, 0)")
    
    div = driver.find_element('xpath', '//*[@id="islrg"]/div[1]')
    img_list = div.find_elements(By.CSS_SELECTOR, ".rg_i.Q4LuWd")
#     img_list = div.find_elements('xpath', '//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]/img')
    print(len(img_list))
    img_list_length = len(img_list)
    #######################################################################################

    for index, img in enumerate(img_list) :
        try:
            click_and_save(dir_name, index, img, img_list_length)
        except ElementClickInterceptedException as e :
            print(e)
            driver.execute_script("window.scrollTo(0, window.scrollY+100)")
            click_and_save(dir_name, index, img, img_list_length)
        except NoSuchElementException as e :
            print(e)
            driver.execute_script("window.scrollTo(0, window.scrollY+100)")
            click_and_save(dir_name, index, img, img_list_length)
        except ConnectionResetError as e :
            print(e)
            pass
        except URLError as e :
            print(e)
            pass
        except socket.timeout as e :
            print(e)
            pass
        except socket.gaierror as e :
            print(e)
            pass
        except ElementNotInteractableException as e:
            print(e)
            break
            
    try :
        print("[스크래핑 종료 (성공률: {:.2f})]".format((scraped_count / img_list_length) * 100))
    except ZeroDivisionError as e :
        print(e)
    
    driver.quit()
    
def resize_and_remove(dir_name, query, filter_size):
    # 일정 해상도 이하이거나 손상된 이미지 제거
    filtered_count = 0
    
    for index, file_name in enumerate(os.listdir(dir_name)):
        try :
            file_path = os.path.join(dir_name, file_name)
            img = cv2.imread(file_path)
            height = img.shape[0]
            width = img.shape[1]
            if height >= width and width > 512 :
                img = cv2.resize(img, (512, int(512/height*width)), interpolation=cv2.INTER_AREA)
                cv2.imwrite(file_path, img)
                print(file_path, ' resized')
            elif width > height and height > 512 :
                img = cv2.resize(img, dsize=(int(512/width*height), 512), interpolation=cv2.INTER_AREA)
                cv2.imwrite(file_path, img)
                print(file_path, ' resized')
                
        except OSError as e :
            print(e)
            os.remove(file_path)
            filtered_count += 1 
        except AttributeError as e:
            print(e)
            os.remove(file_path)
            filtered_count += 1 
        except TypeError as e:
            print(e)
            print(img.shape)
    print(f"[이미지 제거 개수: {filtered_count}/{scraped_count}]")

    
#### 크롤링 키워드 기록
query_list = ['flooding', 'flooding road', '강남역 침수','침수', 
              '道路浸水', '車の洪水', # 중국어, 일본어 (도로침수, 차 침수)
             'Überschwemmungen auf der Straße', # 독일어, 도로 침수
             'inondation des routes', # 프랑스어, 도로 침수
              'inundación de coches', # 스페인어, 차 침수
              'flooded', '침수 cctv',
              
              ### 자동차 포커스 키워드
              'flooded car',
              'überflutetes Auto', # 독일어
             '被淹的车', #중국어
             'coche inundado', # 스페인어
             'voiture inondée', # 프랑스어
             ]

In [7]:
# execute
socket.setdefaulttimeout(30)
driver = webdriver.Chrome(chromedriver_path)

scraped_count = 0

path = '../data/binary/'
#query = input('검색어 입력: ')
query = '도로'
dir_name = path+query
create_folder(dir_name)
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query)
print('##################### 크롤링 끝 #############################')
# resize_and_remove(dir_name, query, 512)

/var/folders/63/8nyxhd_90vg4rctvs_pfm8440000gn/T/ipykernel_44978/3213224274.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_path)


[../data/binary/도로 디렉토리 생성]
[scroll_down() : 스크롤 다운 시작]
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]
[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
[스크롤 다운: 8]
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="islmp"]/div/div/div/div[2]/span"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x00000001027d2f44 chromedriver + 3813188
1   chromedriver                        0x0000000102764ae0 chromedriver + 3361504
2   chromedriver                        0x0000000102468c5c chromedriver + 232540
3   chromedriver                        0x000000010249a380 chromedriver + 435072
4   chromedriver                        0x00000001024c7f44 chromedriver + 622404
5   chromedriver                        0x000000010248f358 chromedriver + 389976
6   chromedriver                        0x0000000102490284 chromedriver + 393860
7   chromedriver                        0x00000001027a6e18 chromedriver + 3632664
8   chro

263/899 JPG 이미지 저장
264/899 JPG 이미지 저장
265/899 JPG 이미지 저장
266/899 JPG 이미지 저장
267/899 JPG 이미지 저장
268/899 JPG 이미지 저장
269/899 JPG 이미지 저장
270/899 JPG 이미지 저장
271/899 JPG 이미지 저장
272/899 JPG 이미지 저장
273/899 PNG 이미지 저장
274/899 JPG 이미지 저장
275/899 JPG 이미지 저장
276/899 JPG 이미지 저장
277/899 JPG 이미지 저장
278/899 JPG 이미지 저장
279/899 JPG 이미지 저장
280/899 JPG 이미지 저장
281/899 JPG 이미지 저장
282/899 JPG 이미지 저장
283/899 JPG 이미지 저장
284/899 JPG 이미지 저장
285/899 JPG 이미지 저장
286/899 JPG 이미지 저장
287/899 JPG 이미지 저장
288/899 JPG 이미지 저장
289/899 JPG 이미지 저장
290/899 JPG 이미지 저장
291/899 JPG 이미지 저장
292/899 JPG 이미지 저장
293/899 JPG 이미지 저장
294/899 JPG 이미지 저장
295/899 JPG 이미지 저장
296/899 JPG 이미지 저장
297/899 JPG 이미지 저장
298/899 JPG 이미지 저장
299/899 JPG 이미지 저장
300/899 JPG 이미지 저장
301/899 JPG 이미지 저장
302/899 JPG 이미지 저장
303/899 JPG 이미지 저장
304/899 JPG 이미지 저장
305/899 JPG 이미지 저장
306/899 JPG 이미지 저장
307/899 JPG 이미지 저장
308/899 JPG 이미지 저장
309/899 JPG 이미지 저장
310/899 JPG 이미지 저장
311/899 JPG 이미지 저장
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify fa

663/899 JPG 이미지 저장
664/899 JPG 이미지 저장
665/899 JPG 이미지 저장
666/899 JPG 이미지 저장
667/899 JPG 이미지 저장
HTTP Error 403: Forbidden
669/899 JPG 이미지 저장
670/899 JPG 이미지 저장
671/899 JPG 이미지 저장
672/899 JPG 이미지 저장
673/899 JPG 이미지 저장
674/899 PNG 이미지 저장
675/899 JPG 이미지 저장
676/899 JPG 이미지 저장
677/899 JPG 이미지 저장
678/899 JPG 이미지 저장
679/899 JPG 이미지 저장
680/899 JPG 이미지 저장
681/899 JPG 이미지 저장
682/899 JPG 이미지 저장
683/899 JPG 이미지 저장
684/899 PNG 이미지 저장
685/899 JPG 이미지 저장
686/899 JPG 이미지 저장
687/899 JPG 이미지 저장
688/899 JPG 이미지 저장
689/899 JPG 이미지 저장
690/899 JPG 이미지 저장
691/899 JPG 이미지 저장
692/899 JPG 이미지 저장
693/899 JPG 이미지 저장
694/899 JPG 이미지 저장
695/899 JPG 이미지 저장
696/899 JPG 이미지 저장
697/899 JPG 이미지 저장
698/899 JPG 이미지 저장
699/899 JPG 이미지 저장
700/899 JPG 이미지 저장
701/899 JPG 이미지 저장
702/899 JPG 이미지 저장
703/899 JPG 이미지 저장
704/899 JPG 이미지 저장
705/899 JPG 이미지 저장
706/899 JPG 이미지 저장
707/899 JPG 이미지 저장
708/899 JPG 이미지 저장
709/899 JPG 이미지 저장
710/899 JPG 이미지 저장
711/899 JPG 이미지 저장
712/899 JPG 이미지 저장
713/899 JPG 이미지 저장
714/899 JPG 이미지 저장
715/8

# Youtube

- 차 침수 영상 : https://www.youtube.com/channel/UCamiAALTPQs-fU0g9OfaJpQ/videos
- 참고 : https://mustzee.tistory.com/27 , https://stackoverflow.com/questions/47420304/download-video-in-mp3-format-using-pytube , https://butnotforme.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C-%EC%9C%A0%ED%8A%9C%EB%B8%8C-%EA%B3%A0%ED%99%94%EC%A7%88-%EC%98%81%EC%83%81-%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C-%ED%95%98%EA%B8%B0

In [4]:
from pytube import Playlist, YouTube
from moviepy.video.VideoClip import * ## moviepy<=1.0.2
from moviepy.audio.AudioClip import * 
from moviepy.editor import *
from IPython.display import clear_output, Video, Audio

fpath = lambda x: '../data/' + x
output_path = '../data/youtube'

def ydown(url: str, prefix: str = ""):
    yt = YouTube(url)
#     yt.streams.filter(adaptive=True, file_extension="mp4", only_video=True).order_by("resolution").desc().first().download(output_path=output_path, filename_prefix=f"{prefix} ")
    
    vpath = (
        yt.streams.filter(adaptive=True, file_extension="mp4", only_video=True)
        .order_by("resolution")
        .desc()
        .first()
        .download(output_path=fpath("youtube_video/"), filename_prefix=f"{prefix} ")
    )
    apath = (
        yt.streams.filter(adaptive=True, file_extension="mp4", only_audio=True)
        .order_by("abr")
        .desc()
        .first()
        .download(output_path=fpath("youtube_audio/"), filename_prefix=f"{prefix} ")
    )

    v = VideoFileClip(vpath)
    a = AudioFileClip(apath)
            
    v.audio = a
    print(v.fps)
    v.write_videofile(fpath(f"youtube/{vpath.split('/')[-1]}"), fps=v.fps)
#     v.download(DOWNLOAD_FOLDER)

def playlistdown(url: str, prefix: str = ""):
    pl = Playlist(url)

    for v in pl.video_urls:
        try:
            ydown(v, prefix)
        except Exception as e:
            print(e)
            continue
            
#출처: https://butnotforme.tistory.com/entry/파이썬으로-유튜브-고화질-영상-다운로드-하기 [butnotforme:티스토리]

In [6]:
url ='https://www.youtube.com/watch?v=e-H7RP3NcOA&list=PL-Htb7K1_wWSJN7WSlRwfXmBRs2f-sVwc'

In [8]:
ydown('https://www.youtube.com/watch?v=UFTTDabfXaM')

29.97002997002997
Moviepy - Building video ../data/zebal.mp4.
MoviePy - Writing audio in zebalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../data/zebal.mp4



Moviepy - Done !
Moviepy - video ready ../data/zebal.mp4


In [11]:
playlist = 'https://www.youtube.com/watch?v=e-H7RP3NcOA&list=PL-Htb7K1_wWSJN7WSlRwfXmBRs2f-sVwc'
playlistdown(playlist, 'mammoth')

30.0
Moviepy - Building video ../data/youtube_raw/mammoth Rufford Mill Ford Compilation Vehicles vs Flooded Ford  1.mp4.
MoviePy - Writing audio in mammoth Rufford Mill Ford Compilation Vehicles vs Flooded Ford  1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../data/youtube_raw/mammoth Rufford Mill Ford Compilation Vehicles vs Flooded Ford  1.mp4



Moviepy - Done !
Moviepy - video ready ../data/youtube_raw/mammoth Rufford Mill Ford Compilation Vehicles vs Flooded Ford  1.mp4
30.0
Moviepy - Building video ../data/youtube_raw/mammoth Vehicles vs Water Ford Compilation  Rufford & Calverton Water Ford  2.mp4.
MoviePy - Writing audio in mammoth Vehicles vs Water Ford Compilation  Rufford & Calverton Water Ford  2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../data/youtube_raw/mammoth Vehicles vs Water Ford Compilation  Rufford & Calverton Water Ford  2.mp4



t:  36%|███████████████████████▌                                         | 3293/9073 [01:33<03:24, 28.23it/s, now=None]

KeyboardInterrupt: 

In [5]:
ydown('https://www.youtube.com/watch?v=pa_KvXlRzk0')

t:   0%|                                                                | 2/9898 [01:45<145:30:56, 52.94s/it, now=None]

29.97002997002997
Moviepy - Building video ../data/youtube/ 물에 잠긴 차량 시민들도 고립…CCTV로 본 도로 침수 상황  SBS.mp4.
MoviePy - Writing audio in  물에 잠긴 차량 시민들도 고립…CCTV로 본 도로 침수 상황  SBSTEMP_MPY_wvf_snd.mp3



t:   0%|                                                                | 2/9898 [01:49<150:12:49, 54.65s/it, now=None]

MoviePy - Done.
Moviepy - Writing video ../data/youtube/ 물에 잠긴 차량 시민들도 고립…CCTV로 본 도로 침수 상황  SBS.mp4




t:   5%|███▎                                                              | 494/9898 [00:08<02:07, 73.62it/s, now=None]


...

t:  91%|███████████████████████████████████████████████████████████▎     | 9026/9898 [03:01<00:30, 28.28it/s, now=None]


t:  95%|█████████████████████████████████████████████████████████████▌   | 9366/9898 [03:09<00:11, 46.10it/s, now=None]


t:  99%|████████████████████████████████████████████████████████████████▏| 9781/9898 [03:17<00:02, 52.55it/s, now=None]


t:   0%|                                                               | 2/9898 [05:09<425:59:55, 154.97s/it, now=None]

Moviepy - Done !
Moviepy - video ready ../data/youtube/ 물에 잠긴 차량 시민들도 고립…CCTV로 본 도로 침수 상황  SBS.mp4
